# E-mail subject line filter

## Getting the data

In [1]:
!wget -nc http://www.csmining.org/index.php/spam-assassin-datasets.html?file=tl_files/Project_Datasets/SpamAssassin%20data/20050311_spam_2.tar.bz2 ../tmp/spam.tar.bz2
!wget -nc http://www.csmining.org/index.php/spam-assassin-datasets.html?file=tl_files/Project_Datasets/SpamAssassin%20data/20030228_easy_ham.tar.bz2 ../tmp/spam.tar.bz2

--2017-10-22 18:01:08--  http://www.csmining.org/index.php/spam-assassin-datasets.html?file=tl_files/Project_Datasets/SpamAssassin%20data/20050311_spam_2.tar.bz2
Resolviendo www.csmining.org (www.csmining.org)... 202.50.209.39
Conectando con www.csmining.org (www.csmining.org)[202.50.209.39]:80... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 2059029 (2,0M) [application/octet-stream]
Guardando como: “spam-assassin-datasets.html?file=tl_files%2FProject_Datasets%2FSpamAssassin%20data%2F20050311_spam_2.tar.bz2”

spam-assassin-datas 100%[===================>]   1,96M   254KB/s    in 8,8s    

2017-10-22 18:01:18 (228 KB/s) - “spam-assassin-datasets.html?file=tl_files%2FProject_Datasets%2FSpamAssassin%20data%2F20050311_spam_2.tar.bz2” guardado [2059029/2059029]

--2017-10-22 18:01:18--  http://www.csmining.org/index.php/spam-assassin-datasets.html?file=tl_files/Project_Datasets/SpamAssassin%20data/20030228_easy_ham.tar.bz2
Resolviendo www.csmining.org (www.csmini